In [8]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clusterconda inconda install -c conda-forge geopystall -c conda-forge geopying stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
!conda install -c conda-forge html5lib--yes
print('Libraries imported.')


Solving environment: \ | / - \ | / - \ | / - \ | / - \ | / - done

# All requested packages already installed.

Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | done

# All requested packages already installed.

Solving environment: \ failed with initial frozen solve. Retrying with flexible solve.

CondaHTTPError: HTTP 000 CONNECTION FAILED for url <https://conda.anaconda.org/conda-forge/linux-64/repodata.json>
Elapsed: -

An HTTP error occurred when trying to retrieve this URL.
HTTP errors are often intermittent, and a simple retry will get you on your way.
ConnectionError(MaxRetryError("HTTPSConnectionPool(host='conda.anaconda.org', port=443): Max retries exceeded with url: /conda-forge/linux-64/repodata.json (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fd6424cf748>: Failed to establish a new connection: [Errno -2] Name or service not kn

In [9]:
conda install -c conda-forge html5lib 

SyntaxError: invalid syntax (<ipython-input-9-8a3a38e4f465>, line 1)

1 . To import the table (Scrape) from  Wikipedia page and build Dataframe.
 

In [ ]:
df = pd.read_html("https://gpsarab.com/shop11/en/content/12-zip-code-in-jordan")[0]
print(f'The number of (rows,cols) for [df] : {df.shape} ')

In [ ]:
df['City'] = df['City'].apply(lambda x: x.split("|")[0].strip() ).astype(str) 
df['Location'] = df['Location'].apply(lambda x: x.split("|")[0].strip()).astype(str) 
df.head()

2. Ignore cells with a borough that is Not assigned.

In [ ]:
df1 = df[df['City']=='Amman']
df1.head()


To get the geographical coordinates of Torrento

In [ ]:
address = 'Amman'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))




Create a map of Toronto with neighborhoods superimposed on top

In [ ]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df1['Latitude'], 
                                           df1['Longitude'],
                                           df1['City'],
                                           df1['Location']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Define Foursquare Credentials and Version.

In [ ]:
CLIENT_ID = '5T2GGWMMZXTOSAYESR0ZQHUT0PPPY31ZGXMZBIKNCDBALKR3' # your Foursquare ID
CLIENT_SECRET = 'FDPDARQ4AZD3XAV235TPLHGGGUKUMLQOUAMTFOJVIXDOHSAM' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)




x

In [ ]:
# type your answer here

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    df1.loc[1,'Latitude'], 
    df1.loc[1,'Longitude'], 
    radius, 
    LIMIT)
url # display URL

In [ ]:
results = requests.get(url).json()
results


create a function to find venues for each Neighbourhood.

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id=' \
              '{}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
amman_venues = getNearbyVenues(names=df1['Location'],
                                   latitudes=df1['Latitude'],
                                   longitudes=df1['Longitude']
                                  )



Let's check the size of the resulting dataframe

In [ ]:
print(amman_venues.shape)
amman_venues.head()

In [ ]:
amman_venues.groupby('Neighborhood').count()


Find out how many unique categories can be curated from all the returned venues

In [ ]:
print('There are {} uniques categories.'.format(len(amman_venues['Venue Category'].unique())))


Analyze Each Neighborhood.

In [ ]:
# one hot encoding
amman_onehot = pd.get_dummies(amman_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
amman_onehot['Neighborhood'] = amman_venues['Neighborhood'] 

fixed_columns = [col for col in amman_onehot.columns if col != 'Neighborhood' ]
fixed_columns =  ['Neighborhood'] + fixed_columns        
amman_onehot = amman_onehot[fixed_columns]

amman_onehot.head()


Grouping rows by neighborhood.

In [ ]:
amman_onehot.shape

In [ ]:
amman_grouped = amman_onehot.groupby('Neighborhood').mean().reset_index()
amman_grouped


Print each neighborhood


In [ ]:
num_top_venues = 5

for hood in amman_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = amman_grouped[amman_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


Write a function to sort the venues in descending order.

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


Create the new dataframe and display the top 10 venues for each neighborhood.

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = amman_grouped['Neighborhood']

for ind in np.arange(amman_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(amman_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()


Cluster Neighborhoods.

In [ ]:
# set number of clusters
kclusters = 5

amman_grouped_clustering = amman_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(amman_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 


In [ ]:
# add clustering labels
# neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
amman_data = df1
amman_merged = amman_data.drop('Zip Code',1)

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
amman_merged = amman_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Location')
amman_merged = amman_merged.drop([16])
amman_merged.dropna(inplace=True) # check the last columns!
amman_merged.head()

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(amman_merged['Latitude'], amman_merged['Longitude'],
                                  amman_merged['Location'],amman_merged['Cluster Labels'].astype(int)):
    
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

        
map_clusters
